In [1]:
import pandas as pd
import zipfile

In [2]:
zf = zipfile.ZipFile('data/vgsales.csv.zip') 
df = pd.read_csv(zf.open('vgsales.csv'))

FileNotFoundError: [Errno 2] No such file or directory: 'data/vgsales.csv.zip'